In [2]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from ansys.mapdl.core.errors import MapdlRuntimeError
from ansys.mapdl.core import launch_mapdl
import re
import os
import pandas as pd
pd.set_option('display.max_colwidth', 150)

In [3]:
source_dir = 'D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess'

subdir_arr = []
file_name_arr = []
complete_path_arr = []

# Lista de subdiretórios
for subdir in os.listdir(source_dir):
    dir_path = os.path.join(source_dir, subdir)
    print(dir_path)
    if os.path.isdir(dir_path):  # Verifica se é um diretório
        subdir_arr.append(dir_path)  # Adiciona à lista

print('\n')
# Extrair o nome da última pasta do primeiro subdiretório
for subdir in subdir_arr:
    file_name = os.path.basename(subdir)
    print(file_name)
    file_name_arr.append(file_name)

print('\n')
for subdir in subdir_arr:
    file_name = os.path.basename(subdir) + '.db'
    complete_path = os.path.join(subdir, file_name)
    print(complete_path)
    complete_path_arr.append(complete_path)


D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L2T2_k3.55_MS25_SP5
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T4_k0.9_MS25_SP55
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L3T5_k0.766_MS25_SP63
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k1.4_MS25_SP78
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k2.2_MS25_SP77
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T3_k8.6_MS25_SP75
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L4T5_k0.4_MS25_SP97
D:/constructal_automate_analysis_files/ec_2a_phi_0.3_lancanova_2024_reprocess\BL2M1P0.3L5T5_k0.485_MS25_SP128


BL2M1P0.3L2T2_k3.55_MS25_SP5
BL2M1P0.3L3T4_k0.9_MS25_SP55
BL2M1P0.3L3T5_k0.766_MS25_SP63
BL2M1P0.3L4T3_k1.4_MS25_SP78
BL2M1P0.3L4T3_k2.2_

In [3]:
material_yielding_stress = 355
p_u = 3479
MAPDL_RUN_LOCATION='D:/constructal_automate_analysis_files'
MAPDL_START_TIMEOUT=120

In [4]:
mapdl = launch_mapdl(
    port=50052,
    run_location=MAPDL_RUN_LOCATION,
    nproc=4,
    override=True,
    loglevel="INFO",
    start_timeout=MAPDL_START_TIMEOUT,
    cleanup_on_exit=True
)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files


In [27]:
def resume_mapdl_analysis(analysis_dir, analysis_file_name, mapdl):
    print(f'Dir: {analysis_dir}. Filename: {analysis_file_name}')
    mapdl.cwd(analysis_dir)
    mapdl.filname(fname=analysis_file_name, key=0)
    mapdl.resume(fname=analysis_file_name, ext = 'db')

In [1]:
def calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u, flag):
    first_to_last_arr_position = len(mapdl.post_processing.frequency_values)-2-flag
    first_to_last_load = mapdl.post_processing.frequency_values[first_to_last_arr_position]

    second_to_last_arr_position = len(mapdl.post_processing.frequency_values)-3-flag
    second_to_last_load = mapdl.post_processing.frequency_values[second_to_last_arr_position]

    diff_between_loads = first_to_last_load - second_to_last_load
    load_increment = p_u/400

    if diff_between_loads > load_increment:
        return first_to_last_arr_position
    else:
        return second_to_last_arr_position

In [5]:
def calc_load_and_stress(mapdl, p_u):
    position = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u)
    n_u = mapdl.post_processing.frequency_values[position]
    sigma_u = n_u/float(9.8)
    return n_u, sigma_u

In [6]:
def calc_z_deflection(mapdl, p_u):
    position = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u) + 1
    mapdl.post1()
    mapdl.set(lstep=1, sbstep=position)
    negative_z_deflection = min(mapdl.post_processing.nodal_displacement("Z"))
    abs_negative_z_deflection = abs(negative_z_deflection)

    positive_z_deflection = max(mapdl.post_processing.nodal_displacement("Z"))

    if abs_negative_z_deflection > positive_z_deflection:
        z_deflection = abs_negative_z_deflection
    else:
        z_deflection = positive_z_deflection
    mapdl.finish()
    return z_deflection

In [7]:
import re
def extract_sp_from_file_name(text):
    match = re.search(r'SP(\d+)', text)
    if match:
        result = match.group(1)
        return result
    else:
        print("Nenhuma correspondência encontrada.")

In [9]:
def calcular_carga_tensao_deslocamento(mapdl, analysis_dir, analysis_file_name, material_yielding_stress, p_u):
    sp = extract_sp_from_file_name(analysis_file_name)
    resume_mapdl_analysis(analysis_dir, analysis_file_name, mapdl)
    n_u, sigma_u = calc_load_and_stress(mapdl, p_u)
    w_max = calc_z_deflection(mapdl, p_u)
    print(f'SP = {sp}')
    print(f'n_u = {n_u:.2f} N/mm')
    print(f'sigma_u = {sigma_u:.2f} MPa')
    print(f'w_max = {w_max:.4f} mm')

In [10]:
def plot_images(mapdl, analysis_db_path, material_yielding_stress, p_u, position):
    VON_MISES_IMG_SUFFIX = '_von_mises_dist.png'
    W_IMG_SUFFIX = '_w_dist.png'

    von_mises_dist_img_path = analysis_db_path.replace('.db', VON_MISES_IMG_SUFFIX)
    w_dist_img_path = analysis_db_path.replace('.db', W_IMG_SUFFIX)

    mapdl.result.plot_principal_nodal_stress(
        position,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        # screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    mapdl.result.plot_principal_nodal_stress(
        position,
        "SEQV",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,          
        edge_color="black",       
        cmap="jet",              
        rng=[0, material_yielding_stress],
        line_width=1.0,           
        off_screen=True,
        screenshot=von_mises_dist_img_path,
        show_scalar_bar=True,
        # show_displacement=True,
        # displacement_factor=2,
        scalar_bar_args={
            "title": "Tensões (MPa)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )

    mapdl.result.plot_nodal_displacement(
        position,
        "UZ",
        lighting=False,
        cpos="iso",
        background="white",
        text_color="black",
        add_text=False,
        show_edges=True,
        edge_color="black",
        cmap="jet",
        line_width=1.0,
        off_screen=True,
        screenshot=w_dist_img_path,
        show_scalar_bar=True,
        scalar_bar_args={
            "title": "Deslocamento em Z (mm)",
            "vertical": False,
            "fmt":"%8.2f",
            "width": 0.4,
            "position_x": 0.55,
            "position_y": 0.15,
            "title_font_size": 20,
            "label_font_size": 16,
        }
    )
    

    return von_mises_dist_img_path, w_dist_img_path

In [11]:
def gerar_graficos_para_analises_com_problema(mapdl, analysis_dir, analysis_file_name, analysis_db_path, material_yielding_stress, p_u):
    resume_mapdl_analysis(analysis_dir, analysis_file_name, mapdl)
    position = calc_post_processing_frequency_values_arr_position_for_ultimate_load(mapdl, p_u)
    plot_images(mapdl, analysis_db_path, material_yielding_stress, p_u, position)

In [66]:
print(file_name_arr)

['BL2M1P0.3L2T2_k3.55_MS25_SP5', 'BL2M1P0.3L3T4_k0.9_MS25_SP55', 'BL2M1P0.3L3T5_k0.766_MS25_SP63', 'BL2M1P0.3L4T3_k1.4_MS25_SP78', 'BL2M1P0.3L4T3_k2.2_MS25_SP77', 'BL2M1P0.3L4T3_k8.6_MS25_SP75', 'BL2M1P0.3L4T5_k0.4_MS25_SP97', 'BL2M1P0.3L5T5_k0.485_MS25_SP128']


In [75]:
flag = 1
analysis_dir = 'D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71'
analysis_file_name = 'BL2M1P0.3L4T2_k1.233_MS25_SP71'

corrigir_dados_com_problema(mapdl, analysis_dir, analysis_file_name, 355, 3479, flag)

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - Current working directory switched to D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - CURRENT JOBNAME REDEFINED AS BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - RESUME ANSYS DATA FROM FILE NAME=BL2M1P0.3L4T2_k1.233_MS25_SP71.db                                                                                                                                                                                                                                   

 *** MAPDL GLOBAL STATUS *** 

 TITLE = BL2M1P0.3L4T2

Dir: D:/constructal_automate_analysis_files/ec_2a_2_2_phi_0.3_lancanova_2024/BL2M1P0.3L4T2_k1.233_MS25_SP71. Filename: BL2M1P0.3L4T2_k1.233_MS25_SP71


INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   03:02:43  JAN 10, 2025 CP=      0.156

 BL2M1P0.3L4T2_k1.233_MS25_SP71                                                



          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

 *** NOTE ***                            CP =       0.156   TIME= 03:02:43
 The nodal coordinates were updated from their original locations during 
 a prior operation (an explicit UPCOORD or UPGEOM, or a linear           
 perturbation analysis).
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - USE LOAD STEP     1  SUBSTEP    26  FOR LOAD CASE 0 

 SET COMMAND GOT LOAD STEP=     1  SUBSTEP=    26  CUMULATIVE ITERATION=   105
   TIME/FREQUENCY=  1459.9    
 TITLE= BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - USE LOAD STEP     1  SUBSTEP    26  FOR LOAD CASE 0 


SP = 71
n_u = 1459.88 N/mm
sigma_u = 148.97 MPa
w_max = 8.5403 mm


In [76]:
mapdl.exit()

INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
INFO - GRPC_127.0.0.1:50052 -  mapdl_core - run - PARAMETER  = D:\constructal_automate_analysis_files\ec_2a_2_2_phi_0.3_lancanova_2024\BL2M1P0.3L4T2_k1.233_MS25_SP71
